# Baseline model

On the task that we never researched before, we need to have a baseline model to be standard of any model that we will explore in the future.

This model is a simple image classification model. The architecture of this model is CNN + FC.

We have only 48 hours for this task. We can't completely spend time on any process. We don't have enough time. For this reason, we explored only pretrain-DenseNet121. We trained this model by using transfer learning.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from pathlib import Path
from tqdm import tqdm_notebook, tqdm, trange
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras.utils import to_categorical
from DataPrep import preprocess_image, augment_pipeline

## Data preprocessing

In [2]:
train_df = pd.read_csv('datasets/train_sets.csv')
val_df = pd.read_csv('datasets/val_sets.csv')

In [3]:
train_df

,Image,Id,whale_id_index
0,66f164af.jpg,new_whale,0
1,81bd5469.jpg,w_143b201,20
2,193ae7a0.jpg,w_5297ab3,95
3,feb15de4.jpg,w_8d46cef,167
4,7fe190ce.jpg,w_43b50e5,76
...,...,...,...
2649,53c207cf.jpg,w_94cd45e,176
2650,8cc05900.jpg,w_7554f44,140
2651,fa8ed4af.jpg,w_43be268,77
2652,59ecd346.jpg,w_2a04ceb,43


In [8]:

class DataGenerator:
    def __init__(self, df, dataset_dir, batch_size=32):
        self.df = df
        self.dataset_dir = dataset_dir
        self.number_class = self.df['whale_id_index'].max() + 1
        self.batch_size = batch_size
    
    @property
    def number_step(self):
        return len(self.df) // self.batch_size
    
    def create_generator(self, shuffle=False, augment=False):
        df = self.df
        if shuffle:
            df = df.sample(frac=1, replace=False)
        while True:
            for i in range(0, len(df), self.batch_size):
                batch_df = df.iloc[i:i+self.batch_size]
                
                xs = []
                ys = []
                for _, whale in batch_df.iterrows():
                    img = cv2.imread(str(self.dataset_dir / whale['Image']))
                    if augment:
                        img = augment_pipeline.augment(image=img)
                    img = preprocess_image(img)
                    
                    _class = to_categorical(whale['whale_id_index'], self.number_class)
                    
                    xs.append(img)
                    ys.append(_class)
                    
                xs = np.array(xs)
                ys = np.array(ys)
                yield xs, ys
                

In [15]:
input_dir = Path('datasets/train/train')

train_datagen = DataGenerator(train_df, input_dir)
train_gen = train_datagen.create_generator(shuffle=True, augment=False)

val_datagen = DataGenerator(val_df, input_dir)
val_gen = val_datagen.create_generator(shuffle=False)

## Model building & Trainning

freeze CNN

In [16]:
backbone = DenseNet121(include_top=False,
                       weights='imagenet',
                       input_shape=(478, 968, 3)
                      )
model = Sequential([
    backbone,
    GlobalAveragePooling2D(),
    Dense(train_datagen.number_class, activation='softmax')
])

In [7]:
backbone.trainable = False

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 15, 30, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 313)               320825    
Total params: 7,358,329
Trainable params: 320,825
Non-trainable params: 7,037,504
_________________________________________________________________


In [9]:
optimizer = Adam(1e-3)
model.compile(optimizer,
              loss='categorical_crossentropy',
              metrics=['acc'],
             )

In [12]:
model_dir = Path('benchmark/baseline-augment')
model_dir.mkdir(exist_ok=True)

model.fit(train_gen,
          steps_per_epoch=train_datagen.number_step,
          validation_data=val_gen,
          validation_steps=val_datagen.number_step,
          epochs=75,
          callbacks=[
              TensorBoard(str(model_dir)),
              ModelCheckpoint(str(model_dir / 'freeze-ep-{epoch:02d}-val_acc{val_acc:.4f}.hdf5'),
                        monitor='val_acc',
                        verbose=1,
                        save_best_only=True,
                        mode='max')
          ]
          
)

Epoch 1/75
82/82 [==============================] - ETA: 0s - loss: 5.2144 - acc: 0.2269
Epoch 00001: val_acc improved from -inf to 0.24688, saving model to benchmark/baseline/freeze-ep-01-val_acc0.2469.hdf5
82/82 [==============================] - 55s 668ms/step - loss: 5.2144 - acc: 0.2269 - val_loss: 4.7075 - val_acc: 0.2469
Epoch 2/75
82/82 [==============================] - ETA: 0s - loss: 4.6283 - acc: 0.2450
Epoch 00002: val_acc did not improve from 0.24688
82/82 [==============================] - 49s 603ms/step - loss: 4.6283 - acc: 0.2450 - val_loss: 4.5046 - val_acc: 0.2469
Epoch 3/75
82/82 [==============================] - ETA: 0s - loss: 4.3387 - acc: 0.2452
Epoch 00003: val_acc did not improve from 0.24688
82/82 [==============================] - 49s 601ms/step - loss: 4.3387 - acc: 0.2452 - val_loss: 4.3367 - val_acc: 0.2469
Epoch 4/75
82/82 [==============================] - ETA: 0s - loss: 4.0773 - acc: 0.2468
Epoch 00004: val_acc did not improve from 0.24688
82/82 [==

Epoch 27/75
82/82 [==============================] - ETA: 0s - loss: 1.3656 - acc: 0.7086
Epoch 00027: val_acc improved from 0.39375 to 0.40625, saving model to benchmark/baseline/freeze-ep-27-val_acc0.4062.hdf5
82/82 [==============================] - 49s 598ms/step - loss: 1.3656 - acc: 0.7086 - val_loss: 2.7969 - val_acc: 0.4062
Epoch 28/75
82/82 [==============================] - ETA: 0s - loss: 1.3123 - acc: 0.7223
Epoch 00028: val_acc improved from 0.40625 to 0.41563, saving model to benchmark/baseline/freeze-ep-28-val_acc0.4156.hdf5
82/82 [==============================] - 49s 594ms/step - loss: 1.3123 - acc: 0.7223 - val_loss: 2.7717 - val_acc: 0.4156
Epoch 29/75
82/82 [==============================] - ETA: 0s - loss: 1.2616 - acc: 0.7338
Epoch 00029: val_acc improved from 0.41563 to 0.41875, saving model to benchmark/baseline/freeze-ep-29-val_acc0.4187.hdf5
82/82 [==============================] - 49s 598ms/step - loss: 1.2616 - acc: 0.7338 - val_loss: 2.7477 - val_acc: 0.418

82/82 [==============================] - 50s 612ms/step - loss: 0.5662 - acc: 0.9241 - val_loss: 2.4523 - val_acc: 0.4875
Epoch 53/75
82/82 [==============================] - ETA: 0s - loss: 0.5489 - acc: 0.9302
Epoch 00053: val_acc improved from 0.48750 to 0.48906, saving model to benchmark/baseline/freeze-ep-53-val_acc0.4891.hdf5
82/82 [==============================] - 50s 606ms/step - loss: 0.5489 - acc: 0.9302 - val_loss: 2.4445 - val_acc: 0.4891
Epoch 54/75
82/82 [==============================] - ETA: 0s - loss: 0.5313 - acc: 0.9317
Epoch 00054: val_acc improved from 0.48906 to 0.49219, saving model to benchmark/baseline/freeze-ep-54-val_acc0.4922.hdf5
82/82 [==============================] - 50s 612ms/step - loss: 0.5313 - acc: 0.9317 - val_loss: 2.4380 - val_acc: 0.4922
Epoch 55/75
82/82 [==============================] - ETA: 0s - loss: 0.5148 - acc: 0.9336
Epoch 00055: val_acc improved from 0.49219 to 0.49375, saving model to benchmark/baseline/freeze-ep-55-val_acc0.4938.hdf

We found it start to overfiting at early epochs and early train acc.
May be this model didn't learn true way that we want.



## Evaluation

In [23]:
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

In [32]:
input_dir = Path('datasets/train/train')

train_df = pd.read_csv('datasets/train_sets.csv')
train_datagen = DataGenerator(train_df, input_dir)
train_gen = train_datagen.create_generator(shuffle=False)

val_df = pd.read_csv('datasets/val_sets.csv')
val_datagen = DataGenerator(val_df, input_dir)
val_gen = val_datagen.create_generator(shuffle=False)

In [6]:
model = load_model('benchmark/baseline/freeze-ep-55-val_acc0.4938.hdf5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 15, 30, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 313)               320825    
Total params: 7,358,329
Trainable params: 320,825
Non-trainable params: 7,037,504
_________________________________________________________________


In [30]:
def evaluate(datagen, gen):
    labels = []
    predictions = []
    for i in tqdm_notebook(range(datagen.number_step)):
        xs, _labels = next(gen)
        _pred = model.predict(xs, verbose=0)

        labels.extend(_labels)
        predictions.extend(_pred)
    labels = np.array(labels).argmax(axis=1)
    predictions = np.array(predictions).argmax(axis=1)
    return labels, predictions

In [33]:
train_labels, train_predictions = evaluate(train_datagen, train_gen)
val_labels, val_predictions = evaluate(val_datagen, val_gen)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


### Note
class 0 is new_whale

In [43]:
print('Train set')
print(classification_report(train_labels, train_predictions))

Train set
              precision    recall  f1-score   support

           0       0.87      0.93      0.90       642
           1       1.00      1.00      1.00         4
           2       1.00      1.00      1.00         6
           3       1.00      0.75      0.86         4
           4       1.00      1.00      1.00         4
           5       1.00      0.75      0.86         4
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         8
           8       1.00      1.00      1.00         4
           9       1.00      1.00      1.00         6
          10       1.00      1.00      1.00         4
          11       1.00      1.00      1.00         5
          12       1.00      1.00      1.00         6
          13       1.00      1.00      1.00        13
          14       1.00      1.00      1.00         4
          15       1.00      1.00      1.00         6
          16       1.00      0.75      0.86         4
          17     

In [44]:
print('Val set')
print(classification_report(val_labels, val_predictions))

Val set
              precision    recall  f1-score   support

           0       0.36      0.86      0.51       158
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           5       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         2
          13       0.60      0.75      0.67         4
          14       0.00      0.00      0.00         1
          15       1.00      1.00      1.00         1
          16       0.00      0.00      0.00         1
          17       

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


On Validation set.
Recall of new whale (class 0) is too much greater than precision. Maybe this model is bias to predict new whale (class 0)

## Conclusion
With a very unbalance dataset, the CNN+FC model seems didn't work well. Whale images of any ID are very similar.
May this model is more suitable with an image classification dataset that obviously class of images.

Some one-shot learning model like Siamese Network may have greater performance than this model because its aim to classify the different feature between 2 images.

In the real-world, this model doesn't suitable because not easy to adapt to a new registered whale.

